In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ideação com modelos generativos na Vertex AI

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Execute no Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral

Ideação é o processo criativo de gerar, desenvolver e comunicar novas ideias. É uma parte fundamental do processo de `design thinking` e pode ser usado para resolver problemas, criar novos produtos ou serviços ou outras tarefas criativas.

Os modelos generativos são uma ferramenta poderosa que pode ser usada para impulsionar a criatividade e a inovação. Ao aprender como usá-los de forma eficaz, você pode melhorar sua capacidade de apresentar novas ideias e soluções para problemas. Uma parte fundamental disso é aprender como estruturar prompts para usar modelos generativos para tarefas de ideação.

Saiba mais sobre o design de prompt na [documentação oficial](https://cloud.google.com/vertex-ai/docs/generative-ai/text/text-overview#prompt_structure).

### Objetivo

Neste tutorial, você aprenderá a usar modelos generativos do Vertex AI SDK para acelerar o processo de criação trabalhando com os seguintes exemplos:
- Geração de campanha de marketing
- Criação de perguntas de compreensão de leitura
- Geração de memes
- Geração de perguntas para entrevistas
- Geração de nomes
- Dicas e conselhos gerais

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Generative AI Studio

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

## Primeiros Passos

### Instalando os SDK da Vertex AI e da Cloud Translate API

In [ ]:
!pip install google-cloud-aiplatform google-cloud-translate --upgrade --user

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [ ]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../../setup-env/README.md).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [ ]:
# import vertexai

# PROJECT_ID = "[seu-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
from vertexai.preview.language_models import TextGenerationModel

#### Carregando o modelo `text-bison`

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

#### Criando a função *wrapper* para utilizar os modelos em Português

Até o momento desde treinamento, as API do Generative AI Studio suportam somente interações no idioma inglês. Para fazermos interações utilizando o idioma português, vamos utilizar a [Cloud Translation API](https://cloud.google.com/translate) para traduzir as nossas solicitações do português para o inglês e as respostas da API de inglês para português.

In [ ]:
from google.cloud import translate

project_id = !gcloud config list project
project_id = project_id[1].split('=')[1].strip()
parent = f'projects/' + project_id


def traduza(texto, idioma_destino):
    client = translate.TranslationServiceClient()

    response = client.translate_text(
        parent=parent,
        contents=[texto],
        target_language_code=idioma_destino,
        mime_type="text/plain"
    )

    return response.translations[0].translated_text

## Exemplos de ideação

### Geração de campanha de marketing

Neste exemplo, nosso exemplo de geração envolverá o processo de criação de uma campanha sobre sustentabilidade. Vamos ver como isso pode ser feito usando a API do PaLM.

In [ ]:
prompt = traduza("Ejecute una campaña de marketing de moda y sostenibilidad.", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text, "es")
)

### Criando questões de compreensão de leitura

Testes de compreensão de leitura são frequentemente usados em escolas e universidades para avaliar as habilidades de leitura de um aluno. Você pode usar a API PaLM para gerar alguns exemplos de perguntas para testar a compreensão de uma pessoa de uma passagem de texto fornecida.

In [ ]:
prompt = traduza("""
Genera 5 preguntas que pongan a prueba la comprensión del lector del siguiente texto.

Texto:
La selva amazónica, también llamada selva amazónica o Amazonas, es un bosque tropical húmedo de hoja ancha en el bioma amazónico que cubre la mayor parte de la cuenca amazónica de América del Sur. Esta cuenca cubre 7.000.000 km2 (2.700.000 millas cuadradas), de los cuales 5.500.000 km2 (2.100.000 millas cuadradas) están cubiertos por selva tropical. Esta región incluye territorios pertenecientes a nueve naciones y 3.344 territorios indígenas formalmente reconocidos.

La mayor parte del bosque, 60%, está en Brasil, seguido de Perú con 13%, Colombia con 10% y cantidades menores en Bolivia, Ecuador, Guayana Francesa, Guayana, Surinam y Venezuela. Cuatro naciones tienen "Amazonas" como el nombre de una de sus regiones administrativas de alto nivel, y Francia usa el nombre "Guyana Amazon Park" para el área protegida de selva tropical de la Guayana Francesa. El Amazonas representa más de la mitad de las selvas tropicales restantes del planeta y comprende la extensión de selva tropical más grande y con mayor biodiversidad del mundo, con alrededor de 390 mil millones de árboles individuales en aproximadamente 16,000 especies.

Más de 30 millones de personas de 350 grupos étnicos diferentes viven en la Amazonía, subdivididos en 9 sistemas políticos nacionales diferentes y 3.344 territorios indígenas formalmente reconocidos. Los pueblos indígenas constituyen el 9% de la población total, y 60 de los grupos permanecen en gran medida aislados.

La selva tropical probablemente se formó durante la época del Eoceno (desde hace 56 millones de años hasta hace 33,9 millones de años). Surgió después de una reducción global de las temperaturas tropicales cuando el Océano Atlántico se expandió lo suficiente como para proporcionar un clima cálido y húmedo a la cuenca del Amazonas. La selva tropical ha existido durante al menos 55 millones de años, y la mayor parte de la región permaneció libre de biomas de tipo sabana al menos hasta la actual edad de hielo, cuando el clima era más seco y la sabana estaba más extendida.

Después del evento de extinción del Cretácico-Paleógeno, la extinción de los dinosaurios y el clima más húmedo pueden haber permitido que la selva tropical se extendiera por todo el continente. Entre 66 y 34 millones de años, la selva tropical se extendió hasta los 45° sur. Las fluctuaciones climáticas en los últimos 34 millones de años han permitido que las regiones de sabana se expandan hacia los trópicos. Durante el Oligoceno, por ejemplo, la selva cubría una franja relativamente estrecha. Se expandió nuevamente durante el Mioceno Medio, luego se retiró a una formación predominantemente tierra adentro en el último máximo glacial. Sin embargo, la selva aún logró prosperar durante estos períodos glaciales, lo que permitió la supervivencia y evolución de una amplia diversidad de especies.

Vista aérea de la selva amazónica
A mediados del Eoceno, se cree que la cuenca del Amazonas fue dividida en la mitad del continente por el Arco del Púrus. El agua en el lado este fluyó hacia el Atlántico, mientras que en el oeste el agua fluyó hacia el Pacífico a través de la cuenca del Amazonas. Sin embargo, a medida que se elevaban las montañas de los Andes, se creó una gran cuenca que rodeaba un lago; conocida hoy como la Cuenca del Solimões. Durante los últimos 5 a 10 millones de años, esta agua acumulada atravesó el Arco de Purus, uniéndose al flujo hacia el este hacia el Atlántico.

Existe evidencia de que ha habido cambios significativos en la vegetación de la selva amazónica durante los últimos 21.000 años hasta el último máximo glacial (LGM) y la subsiguiente deglaciación. Los análisis de los depósitos de sedimentos de los paleolagos de la cuenca del Amazonas y el abanico del Amazonas indican que la precipitación en la cuenca durante el LGM fue menor que en la actualidad, y esto casi con certeza está asociado con una reducción en la cobertura de vegetación tropical húmeda en la cuenca. En estos días, el Amazonas recibe aproximadamente 9 pies de lluvia al año. Sin embargo, existe un debate sobre cuán extensa fue esta reducción. Algunos científicos argumentan que la selva tropical se ha reducido a pequeños refugios aislados separados por bosques abiertos y pastizales; otros científicos argumentan que la selva tropical permaneció prácticamente intacta, pero se extendió menos hacia el norte, el sur y el este de lo que se ve hoy. Este debate ha resultado difícil de resolver porque las limitaciones prácticas de trabajar en la selva tropical significan que el muestreo de datos está sesgado desde el centro de la cuenca del Amazonas, y ambas explicaciones están razonablemente bien respaldadas por los datos disponibles.

Polvo del desierto del Sahara llevado por el viento al Amazonas
Más del 56% del polvo que fertiliza la selva amazónica proviene de la depresión de Bodélé en el norte de Chad, en el desierto del Sahara. El polvo contiene fósforo, que es importante para el crecimiento de las plantas. El polvo anual del Sahara reemplaza la cantidad equivalente de fósforo arrastrado anualmente al suelo amazónico por las lluvias y las inundaciones.

El satélite CALIPSO de la NASA ha medido la cantidad de polvo transportado por el viento desde el Sahara hasta el Amazonas: un promedio de 182 millones de toneladas de polvo son expulsadas del Sahara cada año, a 15 grados de longitud oeste, a lo largo de 2.600 km (1.600 millas) a lo largo del Océano Atlántico (algo de polvo cae en el Atlántico), luego a 35 grados de longitud oeste en la costa este de América del Sur, 27,7 millones de toneladas (15%) de polvo caen sobre la cuenca del Amazonas (22 millones de toneladas consisten en fósforo), 132 millones de toneladas de polvo quedan en el aire, 43 millones de toneladas de polvo son arrastradas por el viento y caen al Mar Caribe, más allá de los 75 grados de longitud oeste.

CALIPSO utiliza un telémetro láser para escanear la atmósfera de la Tierra en busca de la distribución vertical de polvo y otros aerosoles. CALIPSO rastrea regularmente la columna de polvo del Sahara-Amazonas. CALIPSO midió las variaciones en las cantidades de polvo transportado: una caída del 86 % entre la cantidad más alta de polvo transportado en 2007 y la más baja en 2011.
Una posibilidad que causa la variación es el Sahel, una franja de tierra semiárida en la frontera sur del Sahara. Cuando las cantidades de lluvia en el Sahel son mayores, el volumen de polvo es menor. El aumento de las precipitaciones podría hacer que creciera más vegetación en el Sahel, dejando menos arena expuesta a los vientos.[25]

El fósforo del Amazonas también entra en forma de humo debido a la quema de biomasa en África.

Preguntas:
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text, "es")
)

### Geração de memes

Um exemplo de geração de texto mais leve é gerar memes com base em um determinado tópico.

In [ ]:
prompt = traduza("Dame cinco ideas de memes de gatitos:", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=1, top_p=0.8).text, "es")
)

### Geração de perguntas de entrevista

Quer você seja o entrevistador ou o entrevistado, ter algumas perguntas de amostra com as quais você pode trabalhar pode ser muito útil em entrevistas de emprego. Abaixo, usamos a API PaLM para nos ajudar a gerar algumas possíveis perguntas de entrevista para uma função específica.

In [ ]:
prompt = traduza("Genere diez preguntas sobre el cargo de prompt engineer para trabajar con modelos de lenguaje grandes.", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=1, top_p=0.8).text, "es")
)

### Geração de nomes

A geração de nomes é útil em vários cenários, como criar novos personagens para uma história ou nomear um novo produto ou empresa. Você pode gerar ideias para nomes de uma entidade especificada usando a API PaLM.

In [ ]:
prompt = traduza("¿Cuál sería un nombre genial para una floristería que se especializa en vender ramos de flores secas?", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=1, top_p=0.8).text, "es")
)

### Dicas e conselhos gerais

Abaixo está um exemplo de uso da API PaLM para obter dicas e conselhos sobre tópicos gerais.

In [ ]:
prompt = traduza("¿Cuáles serían buenas estrategias para superar el bloqueo del escritor?", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=1, top_p=0.8).text, "es")
)